# 🎬 VideoRobot — نوت‌بوک نهایی Colab
کلون ریپو → نصب وابستگی‌ها → راه‌اندازی بک‌اند **به‌صورت پکیج** → هِلث‌چک با بک‌آف → مانیفست پایدار → رندر با ریتری → دانلود اتمیک MP4 → (اختیاری) تونل عمومی.

نکات:
- اگر ساختار ایمپورت‌ها درست باشد، بدون دستکاری فایل‌ها اجرا می‌شود. اگر نبود، **به‌صورت خودکار** پچ حداقلی اعمال می‌شود.
- خروجی نهایی در `/content` و در صورت اتصال Drive در `MyDrive/VideoRobot/Output` کپی می‌شود.


In [ ]:
# 🧠 Mount Google Drive (optional)from google.colab import drive  # type: ignoreimport pathlib, ostry:    drive.mount('/content/drive')    print('✅ Google Drive mounted.')    DRIVE_ASSETS = pathlib.Path('/content/drive/MyDrive/VideoRobot/Assets')    DRIVE_OUTPUT = pathlib.Path('/content/drive/MyDrive/VideoRobot/Output')    DRIVE_ASSETS.mkdir(parents=True, exist_ok=True)    DRIVE_OUTPUT.mkdir(parents=True, exist_ok=True)    print(f'✅ Assets: {DRIVE_ASSETS}')except Exception as e:    print(f'ℹ️ Could not mount Drive: {e}')    DRIVE_ASSETS = None    DRIVE_OUTPUT = Noneos.environ['PYTHONUNBUFFERED'] = '1'

In [ ]:
# 📦 Clone repository (shallow) + basic toolchain upgradesimport subprocess, pathlib, shutil, sysREPO_URL = 'https://github.com/englishpodcasteasy-glitch/videorobot.git'WORK = pathlib.Path('/content/videorobot')if WORK.exists():    print('🗑️ Cleaning old repo...')    shutil.rmtree(WORK, ignore_errors=True)print('📥 Cloning repository...')subprocess.run(['git','clone','--depth','1',REPO_URL,str(WORK)], check=True)print('✅ Repo cloned at', WORK)print('🔧 Upgrading pip toolchain...')subprocess.run([sys.executable,'-m','pip','install','-q','--upgrade','pip','setuptools','wheel'], check=True)

In [ ]:
# ⚙️ Install system & Python depsimport os, subprocess, sys, pathlibBACKEND_REQ = '/content/videorobot/backend/requirements.txt'!apt-get update -qq && apt-get install -y ffmpeg > /dev/nullif pathlib.Path(BACKEND_REQ).exists():    print('📦 Installing backend requirements...')    subprocess.run([sys.executable,'-m','pip','install','-q','-r',BACKEND_REQ,'--no-cache-dir'], check=True)else:    print('⚠️ requirements.txt not found; installing common deps as fallback')    subprocess.run([sys.executable,'-m','pip','install','-q','flask','flask-cors','moviepy','imageio-ffmpeg','requests'], check=True)

In [ ]:
# 🧩 Prefer clean module import; fallback to minimal patch if neededimport sys, pathlib, reREPO = pathlib.Path('/content/videorobot')BACKEND = REPO / 'backend'sys.path.insert(0, str(REPO))  # allow `import backend`(BACKEND / '__init__.py').write_text('', encoding='utf-8')try:    import backend  # noqa: F401    print('✅ backend package import OK')except Exception as e:    print('⚠️ backend import failed, applying minimal import patch:', e)    patterns = [      (r"\bfrom\s+config\s+import\b",            "from .config import"),      (r"\bfrom\s+scheduler\s+import\b",         "from .scheduler import"),      (r"\bfrom\s+utils\s+import\b",             "from .utils import"),      (r"\bfrom\s+renderer_service\s+import\b",  "from .renderer_service import"),      (r"\bfrom\s+renderer\s+import\b",          "from .renderer import"),      (r"(^|\n)\s*import\s+config(\s|$)",        r"\1from . import config\2"),      (r"(^|\n)\s*import\s+utils(\s|$)",         r"\1from . import utils\2")    ]    changed = []    for py in BACKEND.rglob('*.py'):        txt = py.read_text(encoding='utf-8', errors='ignore'); orig = txt        for pat,rep in patterns: txt = re.sub(pat, rep, txt)        if txt != orig: py.write_text(txt, encoding='utf-8'); changed.append(py.name)    print('🛠️ Patched:', changed if changed else '(no changes needed)')    import importlib; importlib.invalidate_caches()    import backend  # retry    print('✅ backend package import OK after patch')

In [ ]:
# 🚀 Start backend as package and wait for health (exponential backoff)import os, sys, time, subprocess, requests, socket, pathlib, signalREPO = '/content/videorobot'LOG = '/content/backend.log'def kill_stale():    pats = ('backend/main.py','backend.main')    for pat in pats:        for pid in subprocess.getoutput(f"pgrep -f '{pat}' || true").split():            try: os.kill(int(pid), signal.SIGKILL)            except Exception: passkill_stale()def pick_port():    base = int(os.environ.get('BACKEND_PORT','8000'))    for p in (base, base+1, base+2, 7861, 9000):        s = socket.socket()        try:            s.bind(('127.0.0.1', p)); s.close(); return p        except OSError:            pass    return basePORT = pick_port(); os.environ['BACKEND_PORT'] = str(PORT)BASE = f'http://127.0.0.1:{PORT}'print('🚪 BACKEND_PORT =', PORT)env = os.environ.copy(); env['PYTHONUNBUFFERED']='1'with open(LOG,'w',encoding='utf-8') as log_f:    print('📝 Logging to', LOG)    # Build info for reproducibility    subprocess.run(['bash','-lc',f"echo 'Build: ' $(date) >> {LOG}"])    subprocess.run(['bash','-lc',f"python3 --version >> {LOG}"])    subprocess.run(['bash','-lc',f"git -C {REPO} rev-parse HEAD >> {LOG}"])    proc = subprocess.Popen([sys.executable,'-m','backend.main'], cwd=REPO,                            stdout=log_f, stderr=subprocess.STDOUT, text=True, env=env)    print('PID:', proc.pid)backoff = [1,2,4,8,8,8,8,8]healthy = Falsefor i,delay in enumerate(backoff):    for path in ('/healthz','/health','/version'):        try:            r = requests.get(BASE+path, timeout=2)            if r.ok:                print('✅ Health via', path, '→', r.text[:120])                healthy = True; break        except Exception:            pass    if healthy: break    print(f'⏳ Waiting {delay}s for backend...')    time.sleep(delay)if not healthy:    print('\n❌ Backend not healthy. Tail log:\n' + '-'*60)    print(subprocess.getoutput(f'tail -n 200 {LOG} || true'))    raise SystemExit('Backend failed to become healthy.')print('\n✅ Backend ready at', BASE)

In [ ]:
# 📝 Build a deterministic manifest (prefer template if exists)import json, pathlibASSETS = DRIVE_ASSETSTEMPLATE = pathlib.Path('/content/videorobot/examples/manifest_template.json')if TEMPLATE.exists():    manifest = json.loads(TEMPLATE.read_text(encoding='utf-8'))    print('📄 Loaded template manifest')else:    manifest = {      'seed': 7,      'video': {'width': 720, 'height': 1280, 'fps': 30, 'bg_color': '#101318'},      'tracks': [        {'type':'text','content':'سلام دنیا','start':0.2,'duration':3,'x':40,'y':80,'size':72,'color':'#FFFFFF'}      ]    }manifest['seed'] = 7if ASSETS:    vids = sorted(list(ASSETS.glob('*.mp4')))    imgs = sorted(list(ASSETS.glob('*.png')) + list(ASSETS.glob('*.jpg')) + list(ASSETS.glob('*.jpeg')))    auds = sorted(list(ASSETS.glob('*.mp3')) + list(ASSETS.glob('*.wav')) + list(ASSETS.glob('*.m4a')))    if imgs: manifest['tracks'].insert(0, {'type':'image','src':str(imgs[0]),'start':0,'duration':5,'x':0,'y':0,'scale':1.0})    if vids: manifest['tracks'].insert(0, {'type':'video','src':str(vids[0]),'start':0,'fit':'cover'})    if auds: manifest['tracks'].append({'type':'audio','src':str(auds[0]),'start':0,'gain_db':-3})MANIFEST_PATH = pathlib.Path('/content/manifest.json')MANIFEST_PATH.write_text(json.dumps(manifest, ensure_ascii=False, indent=2))print('✅ Manifest saved to', MANIFEST_PATH)

In [ ]:
# 🎥 Render → poll progress (max 15 min) → atomic download MP4import os, json, time, requests, urllib.request, pathlib, sys, subprocess, shutil, tempfilefrom IPython.display import HTML, displayBASE = f"http://127.0.0.1:{os.environ.get('BACKEND_PORT','8000')}"def maybe_fix(text):    t=(text or '').lower()    if any(k in t for k in ['moviepy','imageio-ffmpeg','videofileclip','ffmpeg not found']):        print('⚠️ Server complains about MoviePy/FFmpeg. Installing client helpers…')        subprocess.run([sys.executable,'-m','pip','install','-q','moviepy>=1.0.3,<2','imageio-ffmpeg>=0.4.9,<1'], check=False)        return True    return Falsepayload = json.loads(open('/content/manifest.json','r',encoding='utf-8').read())def post_render(payload, retries=3):    last = None    for attempt in range(1, retries+1):        try:            r = requests.post(BASE + '/render', json=payload, timeout=180)            print(f'POST /render attempt {attempt}:', r.status_code)            try: print(json.dumps(r.json(), ensure_ascii=False, indent=2)[:400])            except Exception: print((r.text or '')[:400])            if r.ok: return r            body = ''            try: body = r.json().get('error') or r.text            except Exception: body = r.text            if maybe_fix(body):                time.sleep(2)        except Exception as e:            last = e; time.sleep(min(5*attempt, 10))    if last: raise last    raise RuntimeError('render failed after retries')r = post_render(payload)r.raise_for_status()js = r.json()job_id = js.get('job_id') or js.get('jobId') or (js.get('data') or {}).get('job_id')assert job_id, '❌ No job_id returned'print('🎬 job_id:', job_id)# poll up to 15 minutesdeadline = time.time() + 15*60tick = 0while time.time() < deadline:    g = requests.get(BASE + f'/progress/{job_id}', timeout=10)    if g.ok:        pj = g.json()        state = pj.get('state') or (pj.get('data') or {}).get('state')        pct   = pj.get('pct')   or (pj.get('data') or {}).get('pct')        msg   = pj.get('message') or (pj.get('data') or {}).get('message')        print(f'tick {tick:03d}:', state, pct, msg or '')        if state == 'success':            break        if state == 'error':            raise SystemExit(f"❌ render failed: {pj}")    else:        print('progress http', g.status_code, (g.text or '')[:200])    tick += 1    time.sleep(1)out = pathlib.Path(f'/content/{job_id}.mp4')tmp = pathlib.Path(tempfile.NamedTemporaryFile(delete=False, suffix='.mp4').name)try:    urllib.request.urlretrieve(BASE + f'/download?jobId={job_id}', str(tmp))    shutil.move(str(tmp), str(out))    print('📦 Downloaded:', out)    if 'DRIVE_OUTPUT' in globals() and DRIVE_OUTPUT:        dst = DRIVE_OUTPUT / out.name        shutil.copy2(str(out), str(dst))        print('✅ Also copied to Google Drive:', dst)except Exception:    cand = pathlib.Path(f'/content/outputs/{job_id}/final.mp4')    if cand.exists():        print('📦 Found:', str(cand))        out = cand    else:        raisedownload_url = BASE + f'/download?jobId={job_id}'display(HTML('<a href="{u}" download target="_blank"><button style="font-size:16px; padding:10px; background-color:#4CAF50; color:white; border:none; border-radius:5px;">🔗 دانلود ویدیو</button></a>'.format(u=download_url)))print('\n🎉 Done! Check', out)

In [ ]:
# 🌩️ (اختیاری) Cloudflare Tunnel برای دسترسی عمومی به بک‌اند# در صورت نیاز اجرا کنید. URL عمومی در خروجی چاپ می‌شود.%%bashset -eif ! command -v cloudflared >/dev/null 2>&1; then  echo '⬇️ Installing cloudflared...'  curl -fsSL https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o /usr/local/bin/cloudflared  chmod +x /usr/local/bin/cloudflaredfiecho '🚀 Starting tunnel to http://127.0.0.1:'${BACKEND_PORT:-8000}cloudflared tunnel --url http://127.0.0.1:${BACKEND_PORT:-8000} --no-autoupdate